#### Raw data source: https://www.pet.gov.tw/PetsMap/PetsMap.aspx

#### Setup

In [39]:
%load_ext rpy2.ipython
%load_ext autoreload
%autoreload 2

%matplotlib inline  
from matplotlib import rcParams
rcParams['figure.figsize'] = (16, 100)

import warnings
from rpy2.rinterface import RRuntimeWarning
warnings.filterwarnings("ignore") # Ignore all warnings
# warnings.filterwarnings("ignore", category=RRuntimeWarning) # Show some warnings

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, HTML
# from playwright.async_api import async_playwright
from bs4 import BeautifulSoup
import requests
import re

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
%%javascript
// Disable auto-scrolling
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [3]:
%%R

require("tidyverse")

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.4.4     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.0
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


Loading required package: tidyverse


#### Playwright scraping 

In [44]:
# "Hey, open up a browser"
playwright = await async_playwright().start()
browser = await playwright.chromium.launch(headless=False)
page = await browser.new_page()
page.set_default_navigation_timeout(100000)

# Dog registration data
years = [2019] #2020, 2021, 2022, 2023]
for year in years:
    await page.goto("https://www.pet.gov.tw/PetsMap/PetsMap.aspx")
    await page.locator("label").filter(has_text="犬").locator("span").first.click()
    await page.locator("#st").fill(f'{year}/01/01')
    await page.locator("#ed").fill(f'{year}/12/31')
    await page.locator("#setInput").click()

    # await page.get_by_role("button", name="縣 市").click()
    # await page.get_by_role("button", name="縣 市").click()

    html = await page.content()
    doc = BeautifulSoup(html, "html.parser")

    # tables = doc.find_all()")
    # for row in tables:
    #     print(row.a.text)

    # print(table[0])
    # filename = f"text_dog_{year}.csv"

    # print("Saving as", filename)
    # tables[0].to_csv(filename, index=False)


AttributeError: 'NavigableString' object has no attribute 'a'

#### Cleaning and data analysis

In [26]:
# Create average dog per household in DOG registration data

years = [2019, 2020, 2021, 2022, 2023]
for year in years:
    df = pd.read_csv(f"animal_registration_data/{year}_dog.csv", skipinitialspace = True)
   
    # remove whitespaces
    df = df.dropna()
    # df['city'] = df['city'].str.replace(' ','')

    # change the columns to int
    df['registered'] = df['registered'].astype(int)
    df['household'] = df['household'].astype(int)

    # calculate the average dog per household
    df['average_dog'] = (df['registered'] / df['household']).round(2)

    # save the file 
    df.to_csv(f"animal_registration_data/{year}_dog.csv", index=False)



In [27]:
# Create average cat per household in CAT registration data

years = [2019, 2020, 2021, 2022, 2023]
for year in years:
    df = pd.read_csv(f"animal_registration_data/{year}_cat.csv", skipinitialspace = True)

    # change the columns to int
    df['registered'] = df['registered'].astype(int)
    df['household'] = df['household'].astype(int)

    # calculate the average dog per household
    df['average_dog'] = (df['registered'] / df['household']).round(2)

    # save the file back
    df.to_csv(f"animal_registration_data/{year}_cat.csv", index=False)

In [34]:
# Merged the translated city name to the CAT registration data

years = [2019, 2020, 2021, 2022, 2023]

for year in years:

    # load files
    df = pd.read_csv(f"animal_registration_data/cat/{year}_cat.csv", skipinitialspace = True) # check the filename!!!
    df_city = pd.read_csv("city.csv")

    # merge number files with traslated city name file
    merged = pd.merge(df, df_city, left_on='city', right_on='Mandarin', how='left')

    # drop the duplicated column
    merged.drop(columns=['Mandarin'], inplace=True)

    # print(merged) # > check if the merge is correct

    # save back to the file
    merged.to_csv(f"animal_registration_data/cat/{year}_cat.csv", index=False) # filename!!!

In [37]:
# Merged the translated city name to the DOG registration data

years = [2019, 2020, 2021, 2022, 2023]

for year in years:

    # load files
    df = pd.read_csv(f"animal_registration_data/dog/{year}_dog.csv", skipinitialspace = True) # filename!!!
    df_city = pd.read_csv("city.csv")

    # merge number files with traslated city name file
    merged = pd.merge(df, df_city, left_on='city', right_on='Mandarin', how='left')

    # drop the duplicated column
    merged.drop(columns=['Mandarin'], inplace=True)

    # print(merged) # > check if the merge is correct

    # save back to the file
    merged.to_csv(f"animal_registration_data/dog/{year}_dog.csv", index=False) # filename!!!